In [1]:
from typing import Optional, Tuple, Any
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.init import trunc_normal_


from sam2.modeling.sam2_base import SAM2Base

class SAM2ImageEncoder(nn.Module):
    def __init__(self, sam_model: SAM2Base) -> None:
        super().__init__()
        self.model = sam_model
        self.image_encoder = sam_model.image_encoder
        self.no_mem_embed = sam_model.no_mem_embed

    def forward(self, x: torch.Tensor) -> tuple[Any, Any, Any]:
        backbone_out = self.image_encoder(x)
        backbone_out["backbone_fpn"][0] = self.model.sam_mask_decoder.conv_s0(
            backbone_out["backbone_fpn"][0]
        )
        backbone_out["backbone_fpn"][1] = self.model.sam_mask_decoder.conv_s1(
            backbone_out["backbone_fpn"][1]
        )

        feature_maps = backbone_out["backbone_fpn"][-self.model.num_feature_levels:]
        vision_pos_embeds = backbone_out["vision_pos_enc"][-self.model.num_feature_levels:]

        feat_sizes = [(x.shape[-2], x.shape[-1]) for x in vision_pos_embeds]

        # flatten NxCxHxW to HWxNxC
        vision_feats = [x.flatten(2).permute(2, 0, 1) for x in feature_maps]
        vision_pos_embeds = [x.flatten(2).permute(2, 0, 1) for x in vision_pos_embeds]

        vision_feats[-1] = vision_feats[-1] + self.no_mem_embed

        feats = [feat.permute(1, 2, 0).reshape(1, -1, *feat_size)
                 for feat, feat_size in zip(vision_feats[::-1], feat_sizes[::-1])][::-1]

        return feats[0], feats[1], feats[2]


class SAM2ImageDecoder(nn.Module):
    def __init__(
            self,
            sam_model: SAM2Base,
            multimask_output: bool
    ) -> None:
        super().__init__()
        self.mask_decoder = sam_model.sam_mask_decoder
        self.prompt_encoder = sam_model.sam_prompt_encoder
        self.model = sam_model
        self.multimask_output = multimask_output

    @torch.no_grad()
    def forward(
            self,
            image_embed: torch.Tensor,
            high_res_feats_0: torch.Tensor,
            high_res_feats_1: torch.Tensor,
            point_coords: torch.Tensor,
            point_labels: torch.Tensor
    ):
        sparse_embedding = self._embed_points(point_coords, point_labels)
        self.sparse_embedding = sparse_embedding
        dense_embedding = self.prompt_encoder.no_mask_embed.weight.reshape(1, -1, 1, 1)

        high_res_feats = [high_res_feats_0, high_res_feats_1]
        image_embed = image_embed

        masks, iou_predictions, _, _ = self.mask_decoder.predict_masks(
            image_embeddings=image_embed,
            image_pe=self.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embedding,
            dense_prompt_embeddings=dense_embedding,
            repeat_image=False,
            high_res_features=high_res_feats,
        )

        if self.multimask_output:
            masks = masks[:, 1:, :, :]
            iou_predictions = iou_predictions[:, 1:]
        else:
            masks, iou_predictions = self.mask_decoder._dynamic_multimask_via_stability(masks, iou_predictions)

        masks = torch.clamp(masks, -32.0, 32.0)
        masks = masks > 0.0
        masks = masks.to(torch.float32)
        masks = masks * 255.0

        return masks, iou_predictions

    def _embed_points(self, point_coords: torch.Tensor, point_labels: torch.Tensor) -> torch.Tensor:

        point_coords = point_coords + 0.5

        padding_point = torch.zeros((point_coords.shape[0], 1, 2), device=point_coords.device)
        padding_label = -torch.ones((point_labels.shape[0], 1), device=point_labels.device)
        point_coords = torch.cat([point_coords, padding_point], dim=1)
        point_labels = torch.cat([point_labels, padding_label], dim=1)

        point_coords[:, :, 0] = point_coords[:, :, 0] / self.model.image_size
        point_coords[:, :, 1] = point_coords[:, :, 1] / self.model.image_size

        point_embedding = self.prompt_encoder.pe_layer._pe_encoding(point_coords)
        point_labels = point_labels.unsqueeze(-1).expand_as(point_embedding)

        point_embedding = point_embedding * (point_labels != -1)
        point_embedding = point_embedding + self.prompt_encoder.not_a_point_embed.weight * (
                point_labels == -1
        )

        for i in range(self.prompt_encoder.num_point_embeddings):
            point_embedding = point_embedding + self.prompt_encoder.point_embeddings[i].weight * (point_labels == i)

        return point_embedding

In [2]:
model_type = 'sam2.1_hiera_small' #@param ["sam2_hiera_tiny", "sam2_hiera_small", "sam2_hiera_large", "sam2_hiera_base_plus"]
# input_size = 768 #@param {type:"slider", min:160, max:4102, step:8}
input_size = 1024 # Bad output if anything else (for now)
multimask_output = False

if model_type == "sam2.1_hiera_tiny":
    model_cfg = "configs/sam2.1/sam2.1_hiera_t.yaml"
elif model_type == "sam2.1_hiera_small":
    model_cfg = "configs/sam2.1/sam2.1_hiera_s.yaml"
elif model_type == "sam2.1_hiera_base_plus":
    model_cfg = "configs/sam2.1/sam2.1_hiera_b+.yaml"
elif model_type == "sam2.1_hiera_large":
    model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

In [3]:
import torch
from sam2.build_sam import build_sam2

sam2_checkpoint = f"checkpoints/{model_type}.pt"

sam2_model = build_sam2(model_cfg, sam2_checkpoint, device="cpu")

img=torch.randn(1, 3, input_size, input_size).cpu()

sam2_encoder = SAM2ImageEncoder(sam2_model).cpu()
high_res_feats_0, high_res_feats_1, image_embed = sam2_encoder(img)
print(high_res_feats_0.shape)
print(high_res_feats_1.shape)
print(image_embed.shape)

torch.onnx.export(sam2_encoder,
                  img,
                  f"{model_type}_encoder.onnx",
                  export_params=True,
                  opset_version=16,
                  do_constant_folding=True,
                  input_names = ['image'],
                  output_names = ['high_res_feats_0', 'high_res_feats_1', 'image_embed']
                )

torch.Size([1, 32, 256, 256])
torch.Size([1, 64, 128, 128])
torch.Size([1, 256, 64, 64])


D:\Datasets_and_code\code\realsense_python\render_project\final2\python\onnx_exporter\sam2\sam2\modeling\backbones\utils.py:30: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if pad_h > 0 or pad_w > 0:
D:\Datasets_and_code\code\realsense_python\render_project\final2\python\onnx_exporter\sam2\sam2\modeling\backbones\utils.py:58: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if Hp > H or Wp > W:


In [4]:
"""sam2_decoder = SAM2ImageDecoder(sam2_model, multimask_output=multimask_output).cpu()

embed_dim = sam2_model.sam_prompt_encoder.embed_dim
embed_size = (sam2_model.image_size // sam2_model.backbone_stride, sam2_model.image_size // sam2_model.backbone_stride)
mask_input_size = [4 * x for x in embed_size]
print(embed_dim, embed_size, mask_input_size)

point_coords = torch.randint(low=0, high=input_size, size=(1, 5, 2), dtype=torch.float)
point_labels = torch.randint(low=0, high=1, size=(1, 5), dtype=torch.float)
mask_input = torch.randn(1, 1, *mask_input_size, dtype=torch.float)
has_mask_input = torch.tensor([1], dtype=torch.float)
orig_im_size = torch.tensor([input_size, input_size], dtype=torch.int32)

masks, scores = sam2_decoder(image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels, mask_input, has_mask_input, orig_im_size)


torch.onnx.export(sam2_decoder,
                  (image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels, mask_input, has_mask_input, orig_im_size),
                  f"{model_type}_decoder.onnx",
                  export_params=True,
                  opset_version=16,
                  do_constant_folding=True,
                  input_names = ['image_embed', 'high_res_feats_0', 'high_res_feats_1', 'point_coords', 'point_labels', 'mask_input', 'has_mask_input', 'orig_im_size'],
                  output_names = ['masks', 'iou_predictions'],
                  dynamic_axes = {"point_coords": {0: "num_labels", 1: "num_points"},
                                  "point_labels": {0: "num_labels", 1: "num_points"},
                                  "mask_input": {0: "num_labels"},
                                  "has_mask_input": {0: "num_labels"}
                  }
                )"""

'sam2_decoder = SAM2ImageDecoder(sam2_model, multimask_output=multimask_output).cpu()\n\nembed_dim = sam2_model.sam_prompt_encoder.embed_dim\nembed_size = (sam2_model.image_size // sam2_model.backbone_stride, sam2_model.image_size // sam2_model.backbone_stride)\nmask_input_size = [4 * x for x in embed_size]\nprint(embed_dim, embed_size, mask_input_size)\n\npoint_coords = torch.randint(low=0, high=input_size, size=(1, 5, 2), dtype=torch.float)\npoint_labels = torch.randint(low=0, high=1, size=(1, 5), dtype=torch.float)\nmask_input = torch.randn(1, 1, *mask_input_size, dtype=torch.float)\nhas_mask_input = torch.tensor([1], dtype=torch.float)\norig_im_size = torch.tensor([input_size, input_size], dtype=torch.int32)\n\nmasks, scores = sam2_decoder(image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels, mask_input, has_mask_input, orig_im_size)\n\n\ntorch.onnx.export(sam2_decoder,\n                  (image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_l

In [5]:
"""sam2_decoder = SAM2ImageDecoder(sam2_model, multimask_output=multimask_output).cpu()

embed_dim = sam2_model.sam_prompt_encoder.embed_dim
embed_size = (sam2_model.image_size // sam2_model.backbone_stride, sam2_model.image_size // sam2_model.backbone_stride)
mask_input_size = [4 * x for x in embed_size]
print(embed_dim, embed_size, mask_input_size)

point_coords = torch.randint(low=0, high=input_size, size=(1, 5, 2), dtype=torch.float)
point_labels = torch.randint(low=0, high=1, size=(1, 5), dtype=torch.float)
#mask_input = torch.randn(1, 1, *mask_input_size, dtype=torch.float)
#has_mask_input = torch.tensor([1], dtype=torch.float)

masks, scores = sam2_decoder(image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels)


torch.onnx.export(sam2_decoder,
                  (image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels),
                  f"{model_type}_decoder.onnx",
                  export_params=True,
                  opset_version=16,
                  do_constant_folding=True,
                  input_names = ['image_embed', 'high_res_feats_0', 'high_res_feats_1', 'point_coords', 'point_labels'],
                  output_names = ['masks', 'iou_predictions'],
                  #dynamic_axes = {"point_coords": {1: "num_points"},
                                  #"point_labels": {1: "num_points"},
                  #}
                )"""

'sam2_decoder = SAM2ImageDecoder(sam2_model, multimask_output=multimask_output).cpu()\n\nembed_dim = sam2_model.sam_prompt_encoder.embed_dim\nembed_size = (sam2_model.image_size // sam2_model.backbone_stride, sam2_model.image_size // sam2_model.backbone_stride)\nmask_input_size = [4 * x for x in embed_size]\nprint(embed_dim, embed_size, mask_input_size)\n\npoint_coords = torch.randint(low=0, high=input_size, size=(1, 5, 2), dtype=torch.float)\npoint_labels = torch.randint(low=0, high=1, size=(1, 5), dtype=torch.float)\n#mask_input = torch.randn(1, 1, *mask_input_size, dtype=torch.float)\n#has_mask_input = torch.tensor([1], dtype=torch.float)\n\nmasks, scores = sam2_decoder(image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels)\n\n\ntorch.onnx.export(sam2_decoder,\n                  (image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels),\n                  f"{model_type}_decoder.onnx",\n                  export_params=True,\n                 

In [6]:
sam2_decoder = SAM2ImageDecoder(sam2_model, multimask_output=multimask_output).cpu()

embed_dim = sam2_model.sam_prompt_encoder.embed_dim
embed_size = (sam2_model.image_size // sam2_model.backbone_stride, sam2_model.image_size // sam2_model.backbone_stride)
mask_input_size = [4 * x for x in embed_size]
print(embed_dim, embed_size, mask_input_size)

point_coords = torch.randint(low=0, high=input_size, size=(1, 20, 2), dtype=torch.float)
point_labels = torch.randint(low=0, high=1, size=(1, 20), dtype=torch.float)
#mask_input = torch.randn(1, 1, *mask_input_size, dtype=torch.float)
#has_mask_input = torch.tensor([1], dtype=torch.float)

masks, scores = sam2_decoder(image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels)


torch.onnx.export(sam2_decoder,
                  (image_embed, high_res_feats_0, high_res_feats_1, point_coords, point_labels),
                  f"{model_type}_decoder.onnx",
                  export_params=True,
                  opset_version=16,
                  do_constant_folding=True,
                  input_names = ['image_embed', 'high_res_feats_0', 'high_res_feats_1', 'point_coords', 'point_labels'],
                  output_names = ['masks', 'iou_predictions'],
                )

256 (64, 64) [256, 256]


D:\Datasets_and_code\code\realsense_python\render_project\final2\python\onnx_exporter\sam2\sam2\modeling\sam\mask_decoder.py:203: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert image_embeddings.shape[0] == tokens.shape[0]
D:\Datasets_and_code\code\realsense_python\render_project\final2\python\onnx_exporter\sam2\sam2\modeling\sam\mask_decoder.py:207: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  image_pe.size(0) == 1
C:\Users\rescarab\.conda\envs\sam_export\lib\site-packages\torch\onnx\symbolic_opset9.py:5383: UserWarning: Exporting aten::index op